# Bibliotecas

In [5]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Modelos de Recomendação para clientes com 1 compra

In [3]:
dataset_cliente_1_compra = pd.read_csv('dataset_cliente_1_compra.csv')

In [4]:
display(dataset_cliente_1_compra)

,id_venda,id_cliente,data_compra(a/m/d),hora_compra(h/m/s),id_local_partida_ida,id_local_destino_ida,id_local_partida_volta,id_local_destino_volta,empresa_onibus_ida,empresa_onibus_volta,valor_ticket,qtd_tickets_compra
0,17812b21f6135be002d24fad99d3544126cc571bfbffec...,cliente462909,2016-09-11,11:05:18,Brasília,a8fd7dc0e1bf71874f7bae013d89b07da30f44bdb001c6...,0,0,5d389f5e2e34c6b0bad96581c22cee0be36dcf627cd73a...,1,181.83,1
1,1abaf00e317b85d19192d594f672f7119e5b9d0deab8ac...,cliente462908,2020-12-15,20:10:20,Brasília,a8fd7dc0e1bf71874f7bae013d89b07da30f44bdb001c6...,a8fd7dc0e1bf71874f7bae013d89b07da30f44bdb001c6...,fbb2a73b0bacf3953186a92029e3e9b130373a9ff14494...,5d389f5e2e34c6b0bad96581c22cee0be36dcf627cd73a...,5d389f5e2e34c6b0bad96581c22cee0be36dcf627cd73a...,464.89,2
2,1583b8c593388ea15056127cd636eaaba3fa52a6b20591...,cliente462907,2020-12-09,19:54:42,Curitiba,06b2d82840e43ed8432b3f444de18b57dbe60637c99379...,0,0,bc52dd634277c4a34a2d6210994a9a5e2ab6d33bb4a3a8...,1,563.01,3
3,a6338b86562d29d36a2ed1517f1ca8bce07d7c383b0576...,cliente462906,2019-07-05,18:39:11,Belém,673a620c399f28257798d26d037c352676c031573a2b52...,0,0,86e50149658661312a9e0b35558d84f6c6d3da797f552a...,1,41.48,1
4,a0555186335a8bbac55a63bf5432b5ab7768743818ffef...,cliente462905,2019-02-07,19:27:14,Porto Alegre,Campinas,0,0,68519a9eca55c68c72658a2a1716aac3788c289859d46d...,1,177.36,1
...,...,...,...,...,...,...,...,...,...,...,...,...
311034,b584bb3d8917197195b9bec7ee239d636af9f5b299024f...,cliente301631,2023-10-12,17:24:43,2bb1c90302bea6044d9075a8181b2d9bead6c3f1050c89...,Rio de Janeiro,0,0,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,1,76.78,2
311035,9d76e8bd58ce22bac6c5906163855b1de56fa1db500647...,cliente301634,2019-06-22,10:30:15,São Paulo,Belém,0,0,c17edaae86e4016a583e098582f6dbf3eccade8ef83747...,1,83.88,1
311036,8e843ea3e891282aad64db51a795729908a8b9cae76d10...,cliente301682,2022-12-18,14:48:20,2bb1c90302bea6044d9075a8181b2d9bead6c3f1050c89...,Rio de Janeiro,0,0,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,1,82.48,1
311037,771e73255a790eb6e5eb24aad58fdf504beafb34866e0b...,cliente301662,2022-02-10,13:10:08,São Paulo,3fdba35f04dc8c462986c992bcf875546257113072a909...,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,119.82,1


In [5]:
# 1. Calcula frequência de destino por origem dentro dos clientes de 1 compra
origem_destino_freq = dataset_cliente_1_compra.groupby(['id_local_partida_ida', 'id_local_destino_ida']).size().reset_index(name='freq')

# 2. Pega o destino mais comum para cada origem
destino_mais_comum_por_origem = origem_destino_freq.sort_values('freq', ascending=False).drop_duplicates('id_local_partida_ida')

# 3. Cria o dicionário de recomendação baseado em origem
mapa_recomendacao = dict(zip(destino_mais_comum_por_origem['id_local_partida_ida'], destino_mais_comum_por_origem['id_local_destino_ida']))

# 4. Função de recomendação com fallback
def recomendar_destino_com_base_na_origem(origem):
    if origem in mapa_recomendacao:
        return mapa_recomendacao[origem]
    else:
        # fallback: destino mais comum entre os clientes com 1 compra
        return dataset_cliente_1_compra['id_local_destino_ida'].mode()[0]

# 5. Aplica a função linha a linha no DataFrame `dataset_cliente_1_compra`
dataset_cliente_1_compra['recomen_prox_destino'] = dataset_cliente_1_compra['id_local_partida_ida'].apply(recomendar_destino_com_base_na_origem)




# Modelo de Recomendação para cliente que fizeram de 2 a 20 compras

In [ ]:
# Ordena o dataset por cliente e data/hora da compra
dataset_de2a20_compras = dataset_de2a20_compras.sort_values(by=['id_cliente', 'data_compra(a/m/d)', 'hora_compra(h/m/s)'])

# Pega sequência de destinos de ida para cada cliente
sequences = dataset_de2a20_compras.groupby('id_cliente')['id_local_destino_ida'].apply(list)

# Cria pares de transições (destino_atual -> destino_seguinte) para todos clientes
pairs = []
for seq in sequences:
    for i in range(len(seq)-1):
        pairs.append((seq[i], seq[i+1]))

pairs_df = pd.DataFrame(pairs, columns=['atual', 'seguinte'])

# Cria matriz de transição: para cada destino_atual, quais os destinos_seguinte e frequência
transitions = pairs_df.groupby(['atual', 'seguinte']).size().reset_index(name='freq')

# Normaliza para obter probabilidades de transição
transitions['prob'] = transitions.groupby('atual')['freq'].transform(lambda x: x / x.sum())

# Agora a função que, dado um destino atual, retorna o destino seguinte mais provável
def recomendar_proximo_destino(destino_atual):
    candidatos = transitions[transitions['atual'] == destino_atual]
    if candidatos.empty:
        # Se não houver histórico, recomenda o destino mais frequente geral
        return dataset_de2a20_compras['id_local_destino_ida'].mode()[0]
    else:
        return candidatos.sort_values('prob', ascending=False).iloc[0]['seguinte']

In [ ]:
# Aplica a função para recomendar próximo destino com base no destino atual da compra
dataset_de2a20_compras['recomen_prox_destino'] = dataset_de2a20_compras['id_local_destino_ida'].apply(recomendar_proximo_destino)

# Modelo de recomendação para clientes que fizeram mais de 21 compras

In [3]:
dataset_20emdiante_compras = pd.read_csv('dataset_20emdiante_compras.csv')

In [6]:
# Parâmetros do modelo

GAMMA = 0.90   # decaimento de recência (0<gamma<1) — quanto menor, mais força o recente
ALPHA = 20.0   # suavização Bayesiana — quanto maior, mais puxa pro padrão global
TOPK  = 3      # quantas recomendações retornar por linha

df = dataset_20emdiante_compras.copy()

# Opcional (pode reduzir memória/tempo):
for col in ['id_cliente', 'id_local_destino_ida']:
    if col in df.columns:
        df[col] = df[col].astype('category')

# 1) Ordenar por cliente e tempo
df = df.sort_values(by=['id_cliente', 'data_compra(a/m/d)', 'hora_compra(h/m/s)'])


# 2) Gerar pares (destino_atual -> proximo_destino) com peso de recência

def build_pairs_with_recency(g):
    # g já está ordenado por tempo
    curr = g['id_local_destino_ida'].iloc[:-1].to_numpy()
    nxt  = g['id_local_destino_ida'].iloc[1:].to_numpy()
    n = len(g)
    # Para transições i=0..n-2: peso = GAMMA^(n-2 - i)  (as mais recentes têm expoente 0 => peso 1)
    exps = (n - 2) - np.arange(n - 1)
    weights = np.power(GAMMA, np.maximum(exps, 0))
    out = pd.DataFrame({
        'id_cliente': g['id_cliente'].iloc[0],
        'curr': curr,
        'nxt': nxt,
        'w': weights
    })
    return out

pairs = df.groupby('id_cliente', observed=True, group_keys=False).apply(build_pairs_with_recency)
pairs = pairs.dropna(subset=['curr','nxt'])


# 3) Probabilidades globais P(nxt | curr)

glob = pairs.groupby(['curr','nxt'], observed=True)['w'].sum().reset_index()
glob['glob_sum_curr'] = glob.groupby('curr', observed=True)['w'].transform('sum')
glob['Pglob'] = glob['w'] / glob['glob_sum_curr']
glob = glob.drop(columns=['w'])

# Se precisar de fallback global (curr sem saída), já deixamos os top globais
top_global = (
    pairs.groupby('nxt', observed=True)['w'].sum().nlargest(TOPK).index.tolist()
)


# 4) Probabilidades por cliente P_u(nxt | curr) + massa de evidência n_u(curr)

user = pairs.groupby(['id_cliente','curr','nxt'], observed=True)['w'].sum().reset_index(name='w_user')
user['user_sum_curr'] = user.groupby(['id_cliente','curr'], observed=True)['w_user'].transform('sum')
user['Pu'] = user['w_user'] / user['user_sum_curr']

# Massa de evidência por (cliente, curr)
mass = user[['id_cliente','curr','user_sum_curr']].drop_duplicates()


# 5) Montar candidatos para cada (cliente, curr) a partir do global
#    (cross-join leve por curr) e misturar com o do cliente

# Todas as combinações (cliente, curr) observadas no histórico
uc = pairs[['id_cliente','curr']].drop_duplicates()

# Juntar com glob por "curr" para gerar candidatos globais para cada (cliente,curr)
cand = uc.merge(glob[['curr','nxt','Pglob']], on='curr', how='left')

# Trazer Pu e a massa (n_u) quando existir
cand = cand.merge(user[['id_cliente','curr','nxt','Pu']], on=['id_cliente','curr','nxt'], how='left')
cand = cand.merge(mass, on=['id_cliente','curr'], how='left')

# n_u = evidência do cliente naquele "curr"
cand['n_u'] = cand['user_sum_curr'].fillna(0.0)
cand['Pu'] = cand['Pu'].fillna(0.0)
cand['Pglob'] = cand['Pglob'].fillna(0.0)

# Suavização Bayesiana:
# P*(nxt|curr,cliente) = (n_u/(n_u+ALPHA))*Pu + (ALPHA/(n_u+ALPHA))*Pglob
cand['P_star'] = (
    (cand['n_u'] / (cand['n_u'] + ALPHA)) * cand['Pu']
    + (ALPHA / (cand['n_u'] + ALPHA)) * cand['Pglob']
)


# 6) Top-K por (cliente, curr)

cand = cand.sort_values(['id_cliente','curr','P_star'], ascending=[True, True, False])
topk = cand.groupby(['id_cliente','curr'], observed=True).head(TOPK)

# Construir mapa (cliente,curr) -> lista de recomendações
topk_list = (
    topk.groupby(['id_cliente','curr'])['nxt']
        .apply(list)
        .to_dict()
)

# Fallback por (cliente,curr) sem candidatos (ex.: curr sem transições globais)
def get_topk_for_pair(cid, curr):
    return topk_list.get((cid, curr), top_global)


# 7) Anexar ao dataset linha a linha (top1 e topk)

def pack_list(xs, k=TOPK):
    xs = xs[:k]
    # string amigável pra CSV; mude para lista se preferir
    return '|'.join(map(str, xs))

df['rec_next_topk'] = [
    pack_list(get_topk_for_pair(cid, curr), TOPK)
    for cid, curr in zip(df['id_cliente'].astype(object), df['id_local_destino_ida'].astype(object))
]
df['rec_next_top1'] = df['rec_next_topk'].str.split('|').str[0]

# Resultado final:
dataset_20emdiante_compras_rec = df

C:\Users\GAMER\AppData\Local\Temp\ipykernel_28632\579588072.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pairs = df.groupby('id_cliente', observed=True, group_keys=False).apply(build_pairs_with_recency)


In [7]:
df

,id_venda,id_cliente,data_compra(a/m/d),hora_compra(h/m/s),id_local_partida_ida,id_local_destino_ida,id_local_partida_volta,id_local_destino_volta,empresa_onibus_ida,empresa_onibus_volta,valor_ticket,qtd_tickets_compra,rec_next_topk,rec_next_top1
28118,3dcc6570e76dacdd2b4e1d499837f7e4679a15a175e6a1...,cliente1,2019-01-29,18:04:55,Juiz de Fora,São José dos Campos,0,0,68519a9eca55c68c72658a2a1716aac3788c289859d46d...,1,272.50,1,São José dos Campos|Rio de Janeiro|São Paulo,São José dos Campos
176483,0a0e20fd8db75a2b3da64a215b8aebdb448f4c5eb75df6...,cliente1,2019-02-06,17:27:15,Juiz de Fora,São José dos Campos,0,0,68519a9eca55c68c72658a2a1716aac3788c289859d46d...,1,544.99,2,São José dos Campos|Rio de Janeiro|São Paulo,São José dos Campos
106698,42820204592144137ab94293edb8c93cbda72aef7a510a...,cliente1,2019-02-06,17:50:49,Juiz de Fora,Porto Alegre,0,0,c75cb66ae28d8ebc6eded002c28a8ba0d06d3a78c6b5cb...,1,131.31,1,Porto Alegre|Rio de Janeiro|São Paulo,Porto Alegre
27855,9ccb284c90935b499f09633499fad0ff37e86f6a55bcb5...,cliente1,2019-02-06,17:54:53,Porto Alegre,2aec3694418f35d89c5dc2e4372741f0787910ef288d30...,0,0,71ee45a3c0db9a9865f7313dd3372cf60dca6479d46261...,1,113.29,1,2aec3694418f35d89c5dc2e4372741f0787910ef288d30...,2aec3694418f35d89c5dc2e4372741f0787910ef288d30...
9538,9bec7b00feef0fabfbcec8f9bcb356bc3d01e1011eddb5...,cliente1,2019-02-07,09:02:17,Juiz de Fora,Vitória,0,0,b4bbe448fde336bb6a7d7d765f36d3327c772b845e7b54...,1,1763.98,5,Vitória|Ponta Grossa|3f46bdea034f311a14efe877f...,Vitória
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261975,70115d58c9bf8b7e99d55fc9a98f20e40ecccfd6b4b453...,cliente999,2023-06-20,18:57:15,da9570a2a77104e59f0185cff795f508d23173949032cf...,Belo Horizonte,0,0,dac53c17c250fd4d4d81eaf6d88435676dac1f3f389644...,1,47.52,1,Belo Horizonte|São Paulo|da9570a2a77104e59f018...,Belo Horizonte
41347,16bc5f29d76d059c46206c99230ca209684d6a98f78fdf...,cliente999,2023-07-02,14:21:42,da9570a2a77104e59f0185cff795f508d23173949032cf...,Belo Horizonte,0,0,dac53c17c250fd4d4d81eaf6d88435676dac1f3f389644...,1,47.52,1,Belo Horizonte|São Paulo|da9570a2a77104e59f018...,Belo Horizonte
26205,cb9d455c5302e101a447e81b45f9c8fe47b20eae26f10f...,cliente999,2023-08-22,20:45:59,da9570a2a77104e59f0185cff795f508d23173949032cf...,Belo Horizonte,0,0,dac53c17c250fd4d4d81eaf6d88435676dac1f3f389644...,1,47.52,1,Belo Horizonte|São Paulo|da9570a2a77104e59f018...,Belo Horizonte
160088,401c04c2252ee0d31b35f42df96c9e2db71034c6921167...,cliente999,2024-02-19,17:24:32,da9570a2a77104e59f0185cff795f508d23173949032cf...,Belo Horizonte,2fca346db656187102ce806ac732e06a62df0dbb2829e5...,da9570a2a77104e59f0185cff795f508d23173949032cf...,dac53c17c250fd4d4d81eaf6d88435676dac1f3f389644...,dac53c17c250fd4d4d81eaf6d88435676dac1f3f389644...,99.60,2,Belo Horizonte|São Paulo|da9570a2a77104e59f018...,Belo Horizonte


In [8]:
df = df.rename(columns={'rec_next_topk': 'top_3_recom'})
df = df.rename(columns={'rec_next_top1': 'principal_recom'})

In [10]:
df.to_csv('recomendacao_destino_20oumais_compras.csv',index = False)